In [1]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    

In [37]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

In [23]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_regularizer=regularizers.L2(0.01))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [24]:
train(60,200,3,3)

Epoch 1/60
57/57 - 92s - loss: 1.6702 - accuracy: 0.4745 - val_loss: 1.3908 - val_accuracy: 0.4952 - 92s/epoch - 2s/step
Epoch 2/60
57/57 - 86s - loss: 1.3199 - accuracy: 0.4745 - val_loss: 1.2350 - val_accuracy: 0.4952 - 86s/epoch - 2s/step
Epoch 3/60
57/57 - 86s - loss: 1.2011 - accuracy: 0.4745 - val_loss: 1.1462 - val_accuracy: 0.4952 - 86s/epoch - 2s/step
Epoch 4/60
57/57 - 87s - loss: 1.1116 - accuracy: 0.4873 - val_loss: 1.0390 - val_accuracy: 0.5543 - 87s/epoch - 2s/step
Epoch 5/60
57/57 - 86s - loss: 1.0162 - accuracy: 0.5331 - val_loss: 0.9562 - val_accuracy: 0.5603 - 86s/epoch - 2s/step
Epoch 6/60
57/57 - 86s - loss: 0.9390 - accuracy: 0.5586 - val_loss: 0.8636 - val_accuracy: 0.6845 - 86s/epoch - 2s/step
Epoch 7/60
57/57 - 85s - loss: 0.7891 - accuracy: 0.6757 - val_loss: 0.7296 - val_accuracy: 0.6962 - 85s/epoch - 1s/step
Epoch 8/60
57/57 - 88s - loss: 0.7143 - accuracy: 0.7009 - val_loss: 0.6746 - val_accuracy: 0.7335 - 88s/epoch - 2s/step
Epoch 9/60
57/57 - 85s - loss: 0

In [38]:
#kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None)
def train(epochs_num,batch_len,conv_layers,neur_layers):
    X_trainR,XX,Y_trainR,YY=train_test_split(
    X_train,Y_train, test_size=0.05, random_state=20)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=2560
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None),kernel_regularizer=regularizers.L2(0.01))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=inputs*30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_trainR,Y_trainR,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [39]:
train(60,200,3,3)

Epoch 1/60
54/54 - 106s - loss: 1.1004 - accuracy: 0.4707 - val_loss: 1.0750 - val_accuracy: 0.4734 - 106s/epoch - 2s/step
Epoch 2/60
54/54 - 81s - loss: 1.0625 - accuracy: 0.4750 - val_loss: 1.0581 - val_accuracy: 0.4734 - 81s/epoch - 2s/step
Epoch 3/60
54/54 - 82s - loss: 1.0565 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 82s/epoch - 2s/step
Epoch 4/60
54/54 - 81s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0568 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 5/60
54/54 - 81s - loss: 1.0557 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 6/60
54/54 - 82s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0568 - val_accuracy: 0.4734 - 82s/epoch - 2s/step
Epoch 7/60
54/54 - 81s - loss: 1.0557 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 8/60
54/54 - 81s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0570 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 9/60
54/54 - 81s - loss: